# Adds calculated baselines (see recalculate_baselines.ipynb) to an existing dataset

In [1]:
# Imports
from pathlib import Path
import pickle

import pandas as pd

from janelia_core.dataprocessing.dataset import ROIDataset
from janelia_core.fileio.data_handlers import NDArrayHandler

## Parameters go here

In [2]:
# Location of excel file specifying where the data for each experiment is saved relative to the base folder

ps = dict()

ps['data_loc_file'] = r'Z:\Exchange\Will\bishoplab\projects\keller_drive\keller_vnc\data\EM_volume_experiment_data_locations.xlsx'

# Additional parameters (constant for all datasets) specifying where the data is stored
ps['base_folder'] =r'W:\\SV4'
ps['dataset_folder'] = 'extracted'

# Name of the group the recalculated baselines as for
ps['group_name'] =  'brain_rois_1_5_5'

# Relative folder holding all data (including the new baseline data) for this group of rois
ps['file_folder'] = 'brain_rois_1_5_5'

# Name of file holding baselines
ps['baseline_filename'] = 'baseline_f_1001.h5'

# TS data string to associate with the baselines
ps['baseline_ts_data_str'] = 'bl_brain_rois_1_5_5_1001'

## Read in file specifying location of each dataset

In [3]:
def c_fcn(str):
    return str.replace("'", "")
converters = {0:c_fcn, 1:c_fcn}

data_locs = pd.read_excel(ps['data_loc_file'], header=1, usecols=[1, 2], converters=converters)

## Add baselines for each dataset

In [4]:
n_datasets = len(data_locs)
for d_i in range(n_datasets):
    
    # Get relevant paths
    data_main_folder = data_locs['Main folder'][d_i]
    data_sub_folder = data_locs['Subfolder'][d_i]
    
    # Base save path - this is the one which holds the subfolders holding the flourescence and baseline data
    base_save_dir =  Path(ps['base_folder']) / data_main_folder / data_sub_folder / Path(ps['dataset_folder'])
    dataset_path = base_save_dir / 'dataset.pkl'
    # Load the dataset
    with open(dataset_path, 'rb') as f:
        dataset = ROIDataset.from_dict(pickle.load(f))

        dataset.roi_groups[ps['group_name']]['ts_labels'].append(ps['baseline_ts_data_str'])
        # Get time stamps from some other existing data in this group
        
        ts = dataset.ts_data[dataset.roi_groups[ps['group_name']]['ts_labels'][0]]['ts']
        new_ts_data_entry = {'ts': ts, 'vls': NDArrayHandler(base_save_dir/ps['file_folder'], ps['baseline_filename'] )}
        dataset.ts_data[ps['baseline_ts_data_str']] = new_ts_data_entry
        
        # Save the dataset
        save_file = base_save_dir / 'dataset_test.pkl'
        with open(save_file, 'wb') as f:
            pickle.dump(dataset.to_dict(), f)
